In [ ]:
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install matplotlib
!git clone https://github.com/ultralytics/yolov5.git
!cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 15.05 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!curl -L "https://universe.roboflow.com/ds/nti2pv84mE?key=JaaIxy5baf" > roboflow.zip
!unzip roboflow.zip -d /content/dataset
!curl -L "https://github.com/username/repository/raw/branch/path/to/video.mp4" -o fsd1.mp4


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   910  100   910    0     0   2996      0 --:--:-- --:--:-- --:--:--  3003
100  460M  100  460M    0     0  60.9M      0  0:00:07  0:00:07 --:--:-- 73.7M
Archive:  roboflow.zip
 extracting: /content/dataset/README.dataset.txt  
 extracting: /content/dataset/README.roboflow.txt  
 extracting: /content/dataset/data.yaml  
   creating: /content/dataset/test/
   creating: /content/dataset/test/images/
 extracting: /content/dataset/test/images/amz_00005_jpg.rf.929133a331751660ce441ecd6e7df290.jpg  
 extracting: /content/dataset/test/images/amz_00014_jpg.rf.07df77f72daf197119a816acba0e56ae.jpg  
 extracting: /content/dataset/test/images/amz_00024_jpg.rf.3797ca6c54b7d695ba121715bb621637.jpg  
 extracting: /content/dataset/test/images/amz_00059_jpg.rf.0f45c5c118e3bdb6334a744fb4490a8e.jpg  
 extracting: /content/dataset/test/images/amz

In [ ]:
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)

data['train'] = '/content/dataset/train/images'
data['val'] = '/content/dataset/valid/images'

with open('/content/dataset/data.yaml', 'w') as f:
    yaml.dump(data, f)


In [ ]:
# Change directory back to yolov5 folder if you aren't already there
%cd /content/yolov5

# Run the training command
!python train.py --img 640 --batch 16 --epochs 50 --data /content/dataset/data.yaml --weights yolov5s.pt --cache


/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 22.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-03 17:11:46.329581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 17:11:46.348751: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03

In [ ]:
video_path = 'fsd1.mp4'  # The file is in the current directory
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Cannot open video")


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Assuming video is loaded in 'cap'
ret, frame = cap.read()
if ret:
    # Convert the color from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Show the image
    plt.figure(figsize=(10, 6))
    plt.imshow(frame_rgb)
    plt.title('First Frame')
    plt.axis('off')
    plt.show()
else:
    print("Failed to grab a frame")


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
import time
import torch

# Load the trained model
model_path = '/content/yolov5/runs/train/exp/weights/best.pt'  # Adjust the path as needed
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

# Path to the video file
video_path = 'fsd1.mp4'  # Adjust the path as needed

# Open the video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Cannot open video")

# Define codec and create VideoWriter object to save output
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (int(cap.get(3)), int(cap.get(4))))

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame from BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Inference
        results = model(frame_rgb)

        # Render results back onto the frame
        frame_result = results.render()[0]

        # Convert array to BGR for OpenCV
        frame_bgr = cv2.cvtColor(frame_result, cv2.COLOR_RGB2BGR)
        out.write(frame_bgr)  # Save to output file

        # Display the frame using cv2_imshow instead of cv2.imshow
        cv2_imshow(frame_bgr)
        time.sleep(0.1)  # Pause a bit for each frame to be visible, adjust as necessary

        # Clear output after each display to avoid flooding the output area
        clear_output(wait=True)

finally:
    # Release everything when job is finished
    cap.release()
    out.release()


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [36]:
from google.colab import files
files.download('output.mp4')  # Download the video file to your local system to check

Buffered data was truncated after reaching the output size limit.